In [1]:
import os
import gc

import numpy as np
import pandas as pd

import anndata
import scanpy as sc

In [2]:
count_file = 'AIT21_10Xv3.h5ad'

input_path = '/home/xingjiepan/data/whole_brain/scRNA-seq/20230723/'
annotation_file = '/home/xingjiepan/data/whole_brain/scRNA-seq/20230723/AIT21_annotation.tsv'

annotation_df = pd.read_csv(annotation_file, sep='\t')
annotation_df['cl'] = annotation_df['cl'].astype(str)
subclass_map = {t[0] : t[1] for t in annotation_df[['cl', 'subclass_label']].itertuples(index=False)}

In [3]:
%%time
adata = sc.read_h5ad(os.path.join(input_path, count_file))
adata.obs['subclass_label'] = adata.obs['cl'].map(subclass_map)

# Filter out cells without subclass annotations or annotated as low quality
mask_not_na = ~adata.obs['subclass_label'].isna()
mask_not_lq = adata.obs['subclass_label'] != 'LQ'
adata = adata[mask_not_na & mask_not_lq].copy()
gc.collect()

CPU times: user 2min 4s, sys: 18min 30s, total: 20min 34s
Wall time: 1h 24min 24s


409

In [4]:
%%time
# Generate an adata with raw count
adata_raw = anndata.AnnData(X=adata.layers['rawcount'], obs=adata.obs, var=adata.var)
adata_raw.write_h5ad(count_file, compression='gzip')
adata_raw

CPU times: user 35min 1s, sys: 20.5 s, total: 35min 22s
Wall time: 42min 29s


AnnData object with n_obs × n_vars = 2341350 × 32285
    obs: 'library_prep', 'gene.counts.0', 'doublet_score', 'roi', 'umi.counts', 'method', 'sex', 'external_donor_name', 'age', 'medical_conditions', 'cl', 'subclass_label'
    var: 'gene_symbol', 'gene_identifier'